In [2]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [3]:
##set tracking uri

mlflow.set_tracking_uri("https://127.0.0.1:5000")


In [19]:
##loading data set - petal length, petal width, sepal length, sepal length, sepal width - flowers

X,y = datasets.load_iris(return_X_y= True)

#split data into train and test sets 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#define model hyperparameter
params = {"penalty":"l2","solver" : "newton-cg", "max_iter":1000, "multi_class":"auto", "random_state":8888}

##train model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

c:\git\Projects\MLFlowProjects\venv\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888,
                   solver='newton-cg')

In [20]:
##Prediction on the test set
y_pred = lr.predict(X_test)

In [21]:
accuracy = accuracy_score(y_test, y_pred)

In [22]:
###MLFlow Tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")

#create ML Flow experiment
mlflow.set_experiment("Quick Start MLFlow")

##start MLFLow Run
with mlflow.start_run():
    ##log hyperparmaters
    mlflow.log_params(params)

    ##log accuracy metrics
    mlflow.log_metrics({"accuracy":accuracy})

    ##tag for the run to remind what is this run for
    mlflow.set_tag("Training Infor", "Basic LR model for IRIS data")

    ##Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    ##log the model
    model_info = mlflow.sklearn.log_model(
        sk_model = lr,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name="tracking-quickstart"


    )





Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/05/13 07:48:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 3


🏃 View run victorious-rat-652 at: http://127.0.0.1:5000/#/experiments/934240718161479422/runs/e0e501f71a1949f1b729d6b1d50e6f80
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/934240718161479422


Created version '3' of model 'tracking-quickstart'.


In [24]:
model_info.model_uri

'runs:/e0e501f71a1949f1b729d6b1d50e6f80/iris_model'

In [ ]:
##Inferencing And validating my model

import mlflow
from mlflow.models import Model

model_uri = model_info.model_uri
# The model is logged with an input example
pyfunc_model = mlflow.pyfunc.load_model(model_uri)
input_data = pyfunc_model.input_example

# Verify the model with the provided input data using the logged dependencies.
# For more details, refer to:
# https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
mlflow.models.predict(
    model_uri=model_uri,
    input_data=input_data,
    env_manager="uv",
)

2025/05/13 07:54:44 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
2025/05/13 07:54:44 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


Exception: Unexpected env manager value 'venv'